### 模型构造

##### 继承Module类来构造模型

In [1]:
import torch
from torch import nn

class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Module的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)  # 输出层


    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [2]:
X = torch.rand(2,784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.1579, -0.1466, -0.0113,  0.1467,  0.1366,  0.1101,  0.0454,  0.1935,
          0.0550, -0.1327],
        [-0.0121, -0.2394, -0.0787,  0.0842,  0.2662,  0.1030,  0.1215,  0.1081,
         -0.0164, -0.0536]], grad_fn=<AddmmBackward>)

##### ModuleList

In [12]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10)  for i in range(10)])

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for i, l in enumerate(self.linears):
            print(self.linears[i // 2],l)
            x = self.linears[i // 2](x) + l(x)
        return x

In [13]:
X = torch.rand(2,10)
net=MyModule()
net(X)

Linear(in_features=10, out_features=10, bias=True) Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=10, bias=True) Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=10, bias=True) Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=10, bias=True) Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=10, bias=True) Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=10, bias=True) Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=10, bias=True) Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=10, bias=True) Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=10, bias=True) Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=10, bias=True) Linear(in_features=10, out_feat

tensor([[-0.6802,  0.3471,  0.2387,  0.3886, -0.3333,  0.5348,  0.7816,  0.2072,
         -0.1981,  0.8197],
        [-0.6356,  0.3836,  0.1984,  0.4265, -0.3446,  0.5677,  0.8387,  0.2339,
         -0.2054,  0.8742]], grad_fn=<AddBackward0>)

##### ModuleDict

In [14]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['linear']) # 访问
print(net.output)
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


##### 构造复杂的网络

In [15]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)

        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)

        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()


In [19]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-6.7492, grad_fn=<SumBackward0>)

##### Module类的子类嵌套调用

In [20]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU()) 

    def forward(self, x):
        return self.net(x)

net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-1.2193, grad_fn=<SumBackward0>)